In [ ]:
import pandas as pd
import joblib  # Import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2


# Load the dataset
data = pd.read_excel('/content/dataset.xlsx', engine='openpyxl')

# Define input and output columns
input_columns = [
    'Net Volume', 'Pulp Area', 'Froth surface area', 'Froth thickness', 'Air Flow rate',
    'R_inf Ccp', 'R_inf Gn', 'R_inf Po', 'R_inf Sp', 'k_max Ccp', 'k_max Gn', 'k_maxPo',
    'k_max Sp', 'Entrainement Savassi parameters', 'Total Solids Flow_Feed',
    'Total Liquid Flow_Feed', 'Pulp Volumetric Flow_Feed', 'Solids SG_Feed', 'Pulp SG_Feed',
    'Solids Fraction_Feed', 'Cu_Feed', 'Fe_Feed', 'Pb_Feed', 'Zn_Feed'
]

output_columns = [
    'Air Efficiency', 'Flotation Rate: Cell 1', 'Entrainment: Cell 1',
    'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate',
    'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate',
    'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings',
    'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings',
    'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails',
    'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate'
]

# Standard Scaling Features
scalerX = StandardScaler()
scaled_input = scalerX.fit_transform(data[input_columns])

# Scaling output data
scalerY = StandardScaler()
scaled_output = scalerY.fit_transform(data[output_columns])

# Splitting data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(scaled_input, scaled_output, test_size=0.2, random_state=42)

# Model Definition using the Best Parameters
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
model.add(Dense(len(output_columns)))
model.compile(optimizer='adam', loss='mse')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_rescaled = scalerY.inverse_transform(y_pred)
y_test_rescaled = scalerY.inverse_transform(y_test)

# Calculate R-squared for each output and the average
r2_scores = [r2_score(y_test_rescaled[:, i], y_pred_rescaled[:, i]) for i in range(y_test_rescaled.shape[1])]
average_r2 = sum(r2_scores) / len(r2_scores)
print(f'Average R-squared: {average_r2:.4f}')

# Save the model and scalers
model.save('/content/final_model.h5')
joblib.dump(scalerX, '/content/scalerX_24.joblib')
joblib.dump(scalerY, '/content/scalerY_23.joblib')


Epoch 1/100
77/77 [==============================] - 2s 6ms/step - loss: 1.5490 - val_loss: 1.2262
Epoch 2/100
77/77 [==============================] - 0s 3ms/step - loss: 1.0881 - val_loss: 1.0034
Epoch 3/100
77/77 [==============================] - 0s 3ms/step - loss: 0.8986 - val_loss: 0.8300
Epoch 4/100
77/77 [==============================] - 0s 3ms/step - loss: 0.7563 - val_loss: 0.6856
Epoch 5/100
77/77 [==============================] - 0s 3ms/step - loss: 0.6306 - val_loss: 0.5670
Epoch 6/100
77/77 [==============================] - 0s 3ms/step - loss: 0.5362 - val_loss: 0.4800
Epoch 7/100
77/77 [==============================] - 0s 3ms/step - loss: 0.4528 - val_loss: 0.4093
Epoch 8/100
77/77 [==============================] - 0s 4ms/step - loss: 0.3953 - val_loss: 0.3579
Epoch 9/100
77/77 [==============================] - 0s 3ms/step - loss: 0.3404 - val_loss: 0.3150
Epoch 10/100
77/77 [==============================] - 0s 3ms/step - loss: 0.3023 - val_loss: 0.2853
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['/content/scalerY_23.joblib']